In [22]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from tensorflow import keras
from tensorflow.keras import layers, models
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from PIL import Image

In [23]:
img_gen =  ImageDataGenerator(rotation_range=20,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             rescale=1/255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

In [27]:
img_gen.flow_from_directory('../raw_data/train', target_size=(150, 150, 3))

Found 400 images belonging to 4 classes.


## CNN Model

In [25]:
model = models.Sequential()

model.add(layers.Conv2D(filters = 32, kernel_size = (4, 4), input_shape = (150, 150, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

## Transfer Learning - VGG16

In [30]:
from tensorflow.keras.applications.vgg16 import VGG16

def load_model():
    model = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
    return model

In [31]:
model = load_model()
model.summary()

58900480/58889256 [==============================] - 86s 1us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                             

<img src="https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png">

In [47]:
from tensorflow.keras import optimizers

# transfer
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
flattening_layer = layers.Flatten()
dense_layer = layers.Dense(500, activation = 'relu')
prediction_layer = layers.Dense(4, activation = 'softmax')

model = models.Sequential([
    base_model, 
    flattening_layer,
    dense_layer, 
    prediction_layer
])

    
opt = optimizers.Adam(learning_rate=1e-4)
model.compile(loss='categorical_crossentropy', # Classification with more classes
                  optimizer=opt,
                  metrics=['accuracy'])    

In [48]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_5 (Flatten)         (None, 8192)              0         
                                                                 
 dense_10 (Dense)            (None, 500)               4096500   
                                                                 
 dense_11 (Dense)            (None, 4)                 2004      
                                                                 
Total params: 18,813,192
Trainable params: 18,813,192
Non-trainable params: 0
_________________________________________________________________
